In [6]:
# -*- coding: utf-8 -*-
# isbul.net — TÜM İLANLAR (liste + detay zenginleştirme, JSON-LD destekli)
import os, time, random, json, html, re
import pandas as pd
from bs4 import BeautifulSoup

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

# =================== AYARLAR ===================
BASE_URL     = "https://www.isbul.net/is-ilanlari"
MAX_PAGES    = 50
CSV_PATH     = "isbul_tr_is_ilanlari.csv"
LIST_DELAY   = (0.25, 0.6)
DETAIL_DELAY = (0.6, 1.4)
WAIT         = 18
HEADLESS     = False

PROFILE_DIR = os.path.expanduser("~/uc_profiles/isbul_profile")
os.makedirs(PROFILE_DIR, exist_ok=True)
# =================================================

def start_driver():
    opts = uc.ChromeOptions()
    opts.add_argument("--lang=tr-TR")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--window-size=1280,900")
    if HEADLESS:
        opts.add_argument("--headless=new")
    return uc.Chrome(options=opts)

def soup_now(driver):
    return BeautifulSoup(driver.page_source, "html.parser")

def clean_text(t):
    if not t: return None
    t = html.unescape(t)
    t = re.sub(r"\s+", " ", t).strip()
    return t or None

def first_select(soup_or_el, selectors):
    for sel in selectors:
        try:
            hit = soup_or_el.select_one(sel)
            if hit: return hit
        except: pass
    return None

def click_any(driver, selectors):
    for sel in selectors:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            driver.execute_script("arguments[0].click();", el)
            return True
        except: pass
    return False

def go_to_page(driver, p):
    url = BASE_URL if p == 1 else f"{BASE_URL}?sf={p}"
    driver.get(url)
    WebDriverWait(driver, WAIT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    # çerez/popup kapat
    click_any(driver, [
        "button[aria-label*='Kabul' i]", "button[aria-label*='Accept' i]",
        "button[aria-label='Kapat']", ".cookie-accept", "button.cookie-accept"
    ])
    time.sleep(random.uniform(*LIST_DELAY))

def collect_from_list(driver, page_index):
    # Lazy load için aşağı kay
    for _ in range(6):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(0.12)
    sp = soup_now(driver)

    list_selectors = ["div.job-list", "ul.job-list", "div.list", "div#jobs", "div.ilan-listesi"]
    card_selectors = ["div.job-list > div", "ul.job-list > li", "div.ilan", "div.job-item", "div.list > div", "div.ilan-karti"]

    container = first_select(sp, list_selectors) or sp
    cards = []
    for sel in card_selectors:
        found = container.select(sel)
        if found: cards = found; break
    if not cards:
        cards = [a.parent for a in sp.select("a[href*='/is-ilani/']")]

    rows = []
    for c in cards:
        a = first_select(c, ["a[href*='/is-ilani/']", "a.job-title", "a.title", "h2 a"])
        href = a.get("href") if a else None
        if href and href.startswith("/"):
            href = "https://www.isbul.net" + href
        title = clean_text(a.get_text(" ", strip=True)) if a else None
        if not title:
            h2 = first_select(c, ["h2", ".job-title", ".title"])
            title = clean_text(h2.get_text(" ", strip=True)) if h2 else None
        if title and href:
            rows.append({"Title": title, "Link": href, "Page": page_index})
    return rows

# ---------- JSON-LD parse ----------
def parse_jobposting_jsonld(sp: BeautifulSoup):
    """
    Sayfadaki tüm application/ld+json scriptlerini dolaşır.
    JobPosting şemasından şirket/konum/description çeker.
    """
    company = location = summary = None
    scripts = sp.find_all("script", type=lambda x: x and "ld+json" in x)
    for sc in scripts:
        try:
            raw = sc.string or sc.get_text()
            if not raw: continue
            data = json.loads(raw)
            # bazen list olur
            cand = data if isinstance(data, list) else [data]
            for obj in cand:
                if not isinstance(obj, dict): continue
                t = obj.get("@type") or obj.get("type")
                if (isinstance(t, list) and "JobPosting" in t) or t == "JobPosting":
                    # company
                    org = obj.get("hiringOrganization")
                    if isinstance(org, dict):
                        company = company or clean_text(org.get("name"))
                    # location
                    jobloc = obj.get("jobLocation")
                    if isinstance(jobloc, list) and jobloc:
                        addr = jobloc[0].get("address") if isinstance(jobloc[0], dict) else None
                    elif isinstance(jobloc, dict):
                        addr = jobloc.get("address")
                    else:
                        addr = None
                    if isinstance(addr, dict):
                        location = location or clean_text(addr.get("addressLocality") or addr.get("addressRegion") or addr.get("addressCountry"))
                    # summary/description
                    desc = obj.get("description") or obj.get("responsibilities")
                    if desc:
                        # HTML içeriği sadeleştir
                        summary = summary or clean_text(BeautifulSoup(desc, "html.parser").get_text(" ", strip=True))
                    # erken çıkış: üçünden biri bulunduysa bile devam edip kalanları doldurabiliriz
        except Exception:
            continue
    return company, location, summary

def enrich_with_details(driver, row, retries=2):
    """
    Detay sayfasına girer. Önce JSON-LD, sonra HTML seçicileriyle Company/Location/Summary doldurur.
    """
    attempt = 0
    while attempt <= retries:
        try:
            driver.get(row["Link"])
            WebDriverWait(driver, WAIT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(random.uniform(*DETAIL_DELAY))

            sp = soup_now(driver)

            # 1) JSON-LD
            company, location, summary = parse_jobposting_jsonld(sp)

            # 2) HTML fallback (görsel metin)
            if not company:
                company = clean_text((first_select(sp, [
                    ".firma-adi", ".company", ".ilan-firma", ".job-company",
                    "h2.company-name", "span.company-name", "div.company-name", "a.company"
                ]) or {}).get_text(" ", strip=True) if first_select(sp, [
                    ".firma-adi", ".company", ".ilan-firma", ".job-company",
                    "h2.company-name", "span.company-name", "div.company-name", "a.company"
                ]) else None)

            if not location:
                location = clean_text((first_select(sp, [
                    ".sehir", ".location", "span.city", "span.job-location",
                    "li.location", "div.location", "p.location"
                ]) or {}).get_text(" ", strip=True) if first_select(sp, [
                    ".sehir", ".location", "span.city", "span.job-location",
                    "li.location", "div.location", "p.location"
                ]) else None)

            if not summary:
                desc_el = first_select(sp, [
                    ".ilan-aciklama", ".description", ".job-desc", "div.job-description",
                    "section.job-description", "div#ilan-detay", ".ilan-detay", "article"
                ])
                if desc_el:
                    summary = clean_text(desc_el.get_text(" ", strip=True))

            # uzun özetleri kısalt
            if summary and len(summary) > 500:
                summary = summary[:497] + "…"

            row["Company"]  = company
            row["Location"] = location
            row["Summary"]  = summary
            return row

        except Exception:
            attempt += 1
            time.sleep(0.6)

    # başarısız olursa boş bırak ama satırı döndür
    return row

# ------------------- ÇALIŞTIR -------------------
driver = start_driver()
all_rows, seen = [], set()

try:
    for page in range(1, MAX_PAGES + 1):
        go_to_page(driver, page)
        list_rows = collect_from_list(driver, page)
        fresh = [r for r in list_rows if r["Link"] not in seen]
        for r in fresh: seen.add(r["Link"])

        print(f"Sayfa {page}: bulunan {len(list_rows)}, yeni {len(fresh)}, toplam {len(all_rows)+len(fresh)}")

        if page > 1 and len(list_rows) == 0:
            print("Boş sayfa geldi, durduruyorum.")
            break

        # Detay zenginleştir
        for r in fresh:
            enr = enrich_with_details(driver, r)
            all_rows.append(enr)
            time.sleep(random.uniform(*LIST_DELAY))

        # ara kayıt
        if page % 5 == 0:
            pd.DataFrame(all_rows).to_csv(CSV_PATH, index=False)
            print(f"[Ara kayıt] CSV güncellendi: {CSV_PATH}")

    df = pd.DataFrame(all_rows)
    display(df.head(30))
    df.to_csv(CSV_PATH, index=False)
    print("CSV kaydedildi:", CSV_PATH)
    print(f"Toplam {len(df)} ilan kaydedildi.")

finally:
    try: driver.quit()
    except: pass


Sayfa 1: bulunan 70, yeni 70, toplam 70


/var/folders/_q/p66_q3xn4nvf990hv_wtwb2c0000gn/T/ipykernel_3866/2258732449.py:142: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  summary = summary or clean_text(BeautifulSoup(desc, "html.parser").get_text(" ", strip=True))


Sayfa 2: bulunan 70, yeni 0, toplam 70
Sayfa 3: bulunan 70, yeni 0, toplam 70
Sayfa 4: bulunan 70, yeni 0, toplam 70
Sayfa 5: bulunan 70, yeni 0, toplam 70
[Ara kayıt] CSV güncellendi: isbul_tr_is_ilanlari.csv
Sayfa 6: bulunan 70, yeni 0, toplam 70
Sayfa 7: bulunan 70, yeni 0, toplam 70
Sayfa 8: bulunan 70, yeni 0, toplam 70
Sayfa 9: bulunan 70, yeni 0, toplam 70
Sayfa 10: bulunan 70, yeni 0, toplam 70
[Ara kayıt] CSV güncellendi: isbul_tr_is_ilanlari.csv
Sayfa 11: bulunan 70, yeni 0, toplam 70
Sayfa 12: bulunan 70, yeni 0, toplam 70
Sayfa 13: bulunan 70, yeni 0, toplam 70
Sayfa 14: bulunan 70, yeni 0, toplam 70
Sayfa 15: bulunan 70, yeni 0, toplam 70
[Ara kayıt] CSV güncellendi: isbul_tr_is_ilanlari.csv
Sayfa 16: bulunan 70, yeni 0, toplam 70
Sayfa 17: bulunan 70, yeni 0, toplam 70
Sayfa 18: bulunan 70, yeni 0, toplam 70
Sayfa 19: bulunan 70, yeni 0, toplam 70
Sayfa 20: bulunan 70, yeni 0, toplam 70
[Ara kayıt] CSV güncellendi: isbul_tr_is_ilanlari.csv
Sayfa 21: bulunan 70, yeni 0, to

,Title,Link,Page,Company,Location,Summary
0,Müşteri Temsilcisi,https://www.isbul.net/is-ilani/satis-ve-pazarl...,1,EA GLOBAL SU ARIt.SİST.SAN.VE TİC.LTD ŞTİ,İzmir,"EA GLOBAL firması olarak, su arıtma cihazları ..."
1,Müşteri Temsilcisi,https://www.isbul.net/is-ilani/satis-ve-pazarl...,1,EA GLOBAL SU ARIt.SİST.SAN.VE TİC.LTD ŞTİ,İzmir,"EA GLOBAL firması olarak, su arıtma cihazları ..."
2,Konsolosluk Vize Ve İkamet Başvuru Danışmanı,https://www.isbul.net/is-ilani/vize-ve-ikamet-...,1,Trapasaport Vize Dan.Turz.Sey.Ltd.Şti,Ankara,Pozisyon: Konsolosluk Vize ve İkamet Başvuru D...
3,Konsolosluk Vize Ve İkamet Başvuru Danışmanı,https://www.isbul.net/is-ilani/vize-ve-ikamet-...,1,Trapasaport Vize Dan.Turz.Sey.Ltd.Şti,Ankara,Pozisyon: Konsolosluk Vize ve İkamet Başvuru D...
4,Elektrik-elektronik Mühendisi,https://www.isbul.net/is-ilani/elekt-309864,1,MHYS ENERJİ MÜH.AŞ.,Ankara,"İş ilanı hakkındaMHYS Enerji, geliştirdiği yen..."
5,Elektrik-elektronik Mühendisi,https://www.isbul.net/is-ilani/elekt-309864,1,MHYS ENERJİ MÜH.AŞ.,Ankara,"İş ilanı hakkındaMHYS Enerji, geliştirdiği yen..."
6,Tam Zamanlı Teknisyen,https://www.isbul.net/is-ilani/tam-a-309868,1,SYA OTOMOTİV ARAÇ ÜSTÜ EKİPM. SAN.VE TİC.LTD ŞTİ,İzmir,Kamu ve özel sektöre mobil araç imalatı yapan ...
7,Tam Zamanlı Teknisyen,https://www.isbul.net/is-ilani/tam-a-309868,1,SYA OTOMOTİV ARAÇ ÜSTÜ EKİPM. SAN.VE TİC.LTD ŞTİ,İzmir,Kamu ve özel sektöre mobil araç imalatı yapan ...
8,Ön Muhasebe,https://www.isbul.net/is-ilani/on-muhasebe-309866,1,SENOL SAHIN ILGIN,İstanbul - Avrupa,Şişli Bomonti’deki Hukuk Büromuza Yeni Bir Ön ...
9,Ön Muhasebe,https://www.isbul.net/is-ilani/on-muhasebe-309866,1,SENOL SAHIN ILGIN,İstanbul - Avrupa,Şişli Bomonti’deki Hukuk Büromuza Yeni Bir Ön ...


CSV kaydedildi: isbul_tr_is_ilanlari.csv
Toplam 70 ilan kaydedildi.
